In [77]:
import pickle as pkl
import numpy as np
from tqdm import tqdm
import pandas as pd
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments, pipeline
from nltk.tokenize import sent_tokenize, word_tokenize
import multiprocessing
import nltk
nltk.download('punkt')
import evaluate


[nltk_data] Downloading package punkt to /home/haskari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [78]:
with open('data_original/cnn.pkl', 'rb') as f:
    cnn_chat=pkl.load(f)

In [79]:
len(cnn_chat)

11490

In [80]:
cnn_chat[13]

['Pence signed the Religious Freedom Restoration Act, which allows businesses to discriminate against gays and lesbians',
 'This move has garnered Pence criticism from various quarters',
 'The backlash from the LGBT community is expected to continue in the future']

In [81]:
#number of empty summaries
count=0
bad_index=[]
for idx,item in enumerate(cnn_chat):
    if item == []:
        count+=1
        bad_index.append(idx)
print(count)



286


In [82]:
count=0
for items in cnn_chat:
    # items=nltk.sent_tokenize(items)

    if not items:
        continue
    elif len(items)==3:
        count+=1

print(count)

8070


In [83]:
print(len(cnn_chat))

11490


In [84]:
data = load_dataset("cnn_dailymail", '3.0.0')
article_key = 'article'
summary_key = 'highlights'

data=data['test']

In [85]:
data=data.select(i for i in range(len(data)) 
                 if i not in set(bad_index))

In [86]:
def tokenize(example):

    example["original_article"] = example[article_key]
    example["article"] = nltk.sent_tokenize(example[article_key])
    # if args.dataset == 'news':
    # check=example[summary_key][0]['text']
    # example["highlights"] = nltk.sent_tokenize(check)
    # else:
    example["highlights"] = nltk.sent_tokenize(example[summary_key])
        
    return example

data = data.map(tokenize, num_proc=multiprocessing.cpu_count())

Map (num_proc=48): 100%|████████| 11204/11204 [00:00<00:00, 14675.96 examples/s]


In [87]:
count=0
for summ in data['highlights']:
    count+=len(summ)

print(count)

42481


In [88]:
count=0
for summ in cnn_chat:
    count+=len(summ)

print(count)
    

33884


In [116]:
with open('data_original/xsum_capped_random.pkl', 'rb') as f:
    xsum_chat=pkl.load(f)

In [117]:
#number of empty summaries
count=0
bad_index=[]
for idx,item in enumerate(xsum_chat):
    if item == []:
        count+=1
        bad_index.append(idx)
print(count)


1363


In [118]:
count=0
for items in xsum_chat:
    #items=nltk.sent_tokenize(items)
    if not items:
        continue
    elif len(items)==1:
        count+=1

print(count)

9971


In [119]:
data = load_dataset("xsum")
article_key = 'document'
summary_key = 'summary'

data=data['test']

In [120]:
data=data.select(i for i in range(len(data)) 
                 if i not in set(bad_index))

In [121]:
data = data.map(tokenize, num_proc=multiprocessing.cpu_count())

Map (num_proc=48): 100%|██████████| 9971/9971 [00:00<00:00, 11872.26 examples/s]


In [122]:
count=0
for summ in data['highlights']:
    count+=len(summ)

print(count)

9971


In [123]:
count=0
for summ in xsum_chat:
    count+=len(summ)

print(count)

9971


In [124]:
with open('data_original/news_capped_random.pkl', 'rb') as f:
    news_chat=pkl.load(f)

In [125]:
#number of empty summaries
count=0
bad_index=[]
for idx,item in enumerate(news_chat):
    if item == '':
        count+=1
        bad_index.append(idx)
print(count)


0


In [126]:
count=0
for items in news_chat:
    items=items
    if not items:
        continue
    elif len(items)==1:
        count+=1

print(count)

901


In [127]:
data = load_dataset("argilla/news-summary")
article_key = 'text'
summary_key = 'prediction'
data = DatasetDict({
    'train': data['test'],
    'test': data['train']})

data=data['test']

In [128]:
data=data.select(i for i in range(len(data)) 
                 if i not in set(bad_index))

In [129]:
def tokenize_news(example):
    example["original_article"] = example[article_key]
    example["article"] = nltk.sent_tokenize(example[article_key])

    check=example[summary_key][0]['text']
    example["highlights"] = nltk.sent_tokenize(check)

    return example

In [130]:
data = data.map(tokenize_news, num_proc=multiprocessing.cpu_count())

Map (num_proc=48): 100%|███████████| 1000/1000 [00:00<00:00, 1947.04 examples/s]


In [131]:
count=0
for summ in data['highlights']:
    count+=len(summ)

print(count)

1012


In [132]:
count=0
for summ in news_chat:
    count+=len(summ)

print(count)

901


In [133]:
with open('data_original/reddit_capped_random.pkl', 'rb') as f:
    reddit_chat=pkl.load(f)

In [134]:
#number of empty summaries
count=0
bad_index=[]
for idx,item in enumerate(reddit_chat):
    if item == []:
        count+=1
        bad_index.append(idx)
print(count)

486


In [135]:
count=0
for items in reddit_chat:
    #items=nltk.sent_tokenize(items)
    if not items:
        continue
    elif len(items)==1:
        count+=1

print(count)

3728


In [136]:
data = load_dataset('reddit_tifu', 'long')
article_key = 'documents'
summary_key = 'tldr'
    # 80% train, 20% test + validation
train_testvalid = data['train'].train_test_split(test_size=0.2, seed=42)
# Split the 20% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)
# gather everyone if you want to have a single DatasetDict
data = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

data=data['test']

In [137]:
data=data.select(i for i in range(len(data)) 
                 if i not in set(bad_index))

In [138]:
data = data.map(tokenize, num_proc=multiprocessing.cpu_count())

Map (num_proc=48): 100%|███████████| 3728/3728 [00:00<00:00, 6242.43 examples/s]


In [139]:
count=0
for summ in data['highlights']:
    count+=len(summ)

print(count)

5288


In [140]:
count=0
for summ in reddit_chat:
    count+=len(summ)

print(count)

3728
